In [1]:
import pandas as pd
from io import StringIO
import os

## Read trace

In [2]:
def read_trace(trace_dir, trace_file, worker_id=None):
    trace_filepath = os.path.join(trace_dir, trace_file)
    
    with open(trace_filepath,"r") as rf:
        new_trace = StringIO()
        line = rf.readline()
        while line:
            line_list = line.split()
            if line_list[0] == "[trace]":
                new_trace.write(line)
            line = rf.readline()
       
    new_trace.seek(0)
    df = pd.read_csv(new_trace, sep=" ", names=["InfoType", "EventType", "EventState", "ID", "Time"])
    del df["InfoType"]
    
    if worker_id is not None:
        df.insert(0, "WorkerID", worker_id)
        
    new_trace.close()
    
    return df

### COORD

In [3]:
trace_dir = "/home/posheng/billy_ws/fl_025/paslab/trace/async2_fl_bw2_3rm/100mbps"
coord_trace_file = "coord.txt"
hpc0_trace_file = "hpc0.txt"
hpc1_trace_file = "hpc1.txt"
hpc2_trace_file = "hpc2.txt"
# hpc3_trace_file = "hpc3_trace.txt"

In [4]:
# coord_df = read_trace(trace_dir, coord_trace_file)
# hpc1_df = read_trace(trace_dir, hpc1_trace_file, "alice")
# hpc2_df = read_trace(trace_dir, hpc2_trace_file, "bob")
# hpc3_df = read_trace(trace_dir, hpc3_trace_file, "charlie")

coord_df = read_trace(trace_dir, coord_trace_file)
w0_df = read_trace(trace_dir, hpc0_trace_file, "alice")
w1_df = read_trace(trace_dir, hpc1_trace_file, "bob")
w2_df = read_trace(trace_dir, hpc2_trace_file, "charlie")

In [5]:
hpc1_df = w0_df
hpc2_df = w1_df
hpc3_df = w2_df

In [6]:
worker_dfs = {
    "alice": hpc1_df,
    "bob": hpc2_df,
    "charlie": hpc3_df
}

## Duration

In [7]:
def get_worker_duration(worker_id, worker_dfs, event_type):
    worker_df = worker_dfs[worker_id]
    worker_event_df = worker_df[worker_df["EventType"].str.contains(event_type, regex=False)]
    worker_event_df = worker_event_df.reset_index()
    
    del worker_event_df['index']
    del worker_event_df['ID']
    
    return worker_event_df

In [8]:
def get_coord_duration(coord_df, event_type):
    coord_event_df = coord_df[coord_df["EventType"].str.contains(event_type, regex=False)]
    coord_event_df = coord_event_df.reset_index()
    
    del coord_event_df['index']
    del coord_event_df['ID']
    
    return coord_event_df

In [9]:
def calculate_duration(event_type, df):
    event_df = df[df["EventType"] == event_type][["EventType", "EventState", "ID", "Time"]]
    event_start_df = event_df[event_df["EventState"] == "start"][["ID", "Time"]]
    event_start_df = event_start_df.reset_index()

    event_end_df = event_df[event_df["EventState"] == "end"][["Time"]]
    event_end_df = event_end_df.reset_index()
    
    event_df_cat = pd.concat([event_start_df, event_end_df], axis=1)
    del event_df_cat['index']
    event_df_cat.columns = ["ID", "StartTime", "EndTime"]
    
    event_df_cat["Duration"] = event_df_cat.apply(lambda x: x["EndTime"] - x["StartTime"], axis=1)
    event_df_cat.insert(0, "EventType", event_type)
    return event_df_cat


In [10]:
GlobalModelSendDuration = calculate_duration("GlobalModelSend", coord_df)

In [11]:
LossFuncSendDuration = calculate_duration("LossFuncSend", coord_df)

In [12]:
ModelConfigSendDuration = calculate_duration("ModelConfigSend", coord_df)

### GlobalModelSendDuration

In [13]:
GlobalModelSendDuration[GlobalModelSendDuration["ID"] == "alice"][1:]['Duration'].mean()

18.472403264045717

In [14]:
GlobalModelSendDuration[GlobalModelSendDuration["ID"] == "bob"][1:]['Duration'].mean()

18.242636680603027

In [15]:
GlobalModelSendDuration[GlobalModelSendDuration["ID"] == "charlie"][1:]['Duration'].mean()

18.31258895397186

### LossFuncSendDuration

In [16]:
LossFuncSendDuration[LossFuncSendDuration["ID"] == "alice"][1:]['Duration'].mean()

0.0017537832260131835

In [17]:
LossFuncSendDuration[LossFuncSendDuration["ID"] == "bob"][1:]['Duration'].mean()

0.0015879154205322265

In [18]:
LossFuncSendDuration[LossFuncSendDuration["ID"] == "charlie"][1:]['Duration'].mean()

0.001598525047302246

### ModelConfigSendDuration

In [19]:
ModelConfigSendDuration[ModelConfigSendDuration["ID"] == "alice"][1:]['Duration'].mean()

0.001003122329711914

In [20]:
ModelConfigSendDuration[ModelConfigSendDuration["ID"] == "bob"][1:]['Duration'].mean()

0.0009503006935119629

In [21]:
ModelConfigSendDuration[ModelConfigSendDuration["ID"] == "charlie"][1:]['Duration'].mean()

0.0009734392166137696

## GlobalInformationSend Duration

In [22]:
def get_coord_global_send_duration(coord_df, worker_id):
    coord_event_df = coord_df[coord_df["EventType"].str.contains("GlobalInformationSend", regex=False)]
    coord_event_df = coord_event_df[coord_event_df["ID"] == worker_id]
    coord_event_df = coord_event_df.reset_index()
    
    del coord_event_df['index']
    
    return coord_event_df

In [23]:
1602055807.1283371 - 1602055807.1122975

0.016039609909057617

In [24]:
hpc1_gs_df = get_coord_global_send_duration(coord_df, "alice")

In [25]:
hpc2_gs_df = get_coord_global_send_duration(coord_df, "bob")

In [26]:
hpc3_gs_df = get_coord_global_send_duration(coord_df, "charlie")

In [27]:
hpc1_gs_df['Time']

0      9.382159
1      9.409346
2      9.946156
3      9.417216
4      9.657422
5      9.642013
6      9.270397
7     10.075539
8      9.744531
9      9.704500
10     9.322534
11     9.339094
12    10.123769
13     9.668732
14     9.870749
15     9.661352
16     9.287037
17     9.849598
18     9.943597
19     9.580856
20     9.506539
Name: Time, dtype: float64

In [28]:
hpc1_gs_df['Time'][1:].mean()

9.651048886775971

In [29]:
hpc2_gs_df['Time'][1:].mean()

9.778961324691773

In [30]:
hpc3_gs_df['Time'][1:].mean()

9.70000067949295

## Send Time

In [31]:
## EventType:
##     * GlobalModelSend
##     * LossFuncSend
##     * ModelConfigSend
def calculate_send_time(coord_df, worker_dfs, event_type):
    event_start_df = coord_df[(coord_df["EventType"] == event_type) & (coord_df["EventState"] == "start")]
    worker_event_send_time_dfs = {}
    
    for worker_id in worker_dfs.keys():
        worker_event_start_df = event_start_df[event_start_df["ID"] == worker_id]
        worker_event_start_df = worker_event_start_df.reset_index()
        del worker_event_start_df['index']
        worker_event_start_df.columns = ["EventType", "EventState", "ID", "StartTime"]
        
        worker_df = worker_dfs[worker_id]
        worker_event_recv_df = worker_df[worker_df["EventType"] == event_type]
        worker_event_recv_df = worker_event_recv_df.reset_index()
        del worker_event_recv_df['index']
        worker_event_recv_df.columns = ["WorkerID", "EventType", "EventState", "ID", "RecvTime"]
        
        worker_recv_time_series = worker_event_recv_df["RecvTime"]
        worker_event_send_time_df = pd.concat([worker_event_start_df, worker_recv_time_series], axis=1)
        worker_event_send_time_df["SendTime"] = worker_event_send_time_df.apply(lambda x: x["RecvTime"] - x["StartTime"], axis=1)
        del worker_event_send_time_df["EventState"]
        
        worker_event_send_time_dfs[worker_id] = worker_event_send_time_df
        
    return worker_event_send_time_dfs

In [32]:
worker_gms_send_time_dfs = calculate_send_time(coord_df, worker_dfs, "GlobalModelSend")

In [33]:
worker_gms_send_time_dfs['alice']['SendTime'].mean()

9.644562516893659

In [34]:
worker_gms_send_time_dfs['alice']

,EventType,ID,StartTime,RecvTime,SendTime
0,GlobalModelSend,alice,1.603614e+09,1.603614e+09,9.385497
1,GlobalModelSend,alice,1.603614e+09,1.603614e+09,9.414423
2,GlobalModelSend,alice,1.603614e+09,1.603614e+09,9.952540
3,GlobalModelSend,alice,1.603614e+09,1.603614e+09,9.424333
4,GlobalModelSend,alice,1.603614e+09,1.603614e+09,9.663048
5,GlobalModelSend,alice,1.603615e+09,1.603615e+09,9.648370
6,GlobalModelSend,alice,1.603615e+09,1.603615e+09,9.277519
7,GlobalModelSend,alice,1.603615e+09,1.603615e+09,10.081920
8,GlobalModelSend,alice,1.603615e+09,1.603615e+09,9.751221
9,GlobalModelSend,alice,1.603615e+09,1.603615e+09,9.709507


In [35]:
worker_gms_send_time_dfs['charlie']['SendTime'].mean()

9.729148558207921

In [36]:
# coord_df = read_trace(trace_dir, coord_trace_file)
# hpc1_df = read_trace(trace_dir, hpc1_trace_file, "alice")
# hpc2_df = read_trace(trace_dir, hpc2_trace_file, "bob")
# hpc3_df = read_trace(trace_dir, hpc3_trace_file, "charlie")

## LocalTraining Duration

In [37]:
hpc1_lt_df = get_worker_duration("alice", worker_dfs, "LocalTraining")

In [38]:
hpc2_lt_df = get_worker_duration("bob", worker_dfs, "LocalTraining")

In [39]:
hpc3_lt_df = get_worker_duration("charlie", worker_dfs, "LocalTraining")

In [40]:
hpc1_lt_df['Time'][1:].mean()

26.888133311271666

In [41]:
hpc2_lt_df['Time'][1:].mean()

18.266179859638214

In [42]:
hpc3_lt_df['Time'][1:].mean()

19.399863994121553

In [43]:
hpc1_lt_df

,WorkerID,EventType,EventState,Time
0,alice,LocalTraining,duration,26.827748
1,alice,LocalTraining,duration,26.892097
2,alice,LocalTraining,duration,26.863703
3,alice,LocalTraining,duration,26.661787
4,alice,LocalTraining,duration,26.972345
5,alice,LocalTraining,duration,26.856486
6,alice,LocalTraining,duration,27.062217
7,alice,LocalTraining,duration,26.515647
8,alice,LocalTraining,duration,26.919083
9,alice,LocalTraining,duration,26.799654


## FitSagg Duration

In [44]:
def get_coord_fitsagg_duration(coord_df, worker_id):
    coord_event_df = coord_df[coord_df["EventType"].str.contains("FitSagg", regex=False)]
    coord_event_df = coord_event_df[coord_event_df["ID"] == worker_id]
    coord_event_df = coord_event_df.reset_index()
    
    del coord_event_df['index']
    
    return coord_event_df

In [45]:
hpc1_fs_df = get_coord_fitsagg_duration(coord_df, "alice")

In [46]:
hpc2_fs_df = get_coord_fitsagg_duration(coord_df, "bob")

In [47]:
hpc3_fs_df = get_coord_fitsagg_duration(coord_df, "charlie")

In [48]:
hpc1_fs_df['Time'][1:].mean()

92.98966790437699

In [49]:
hpc2_fs_df['Time'][1:].mean()

98.00929516553879

In [50]:
hpc3_fs_df['Time'][1:].mean()

98.64103366136551

## EncryptMultiply Duration

In [51]:
hpc1_em_df = get_worker_duration("alice", worker_dfs, "EncryptMultiply")

In [52]:
hpc2_em_df = get_worker_duration("bob", worker_dfs, "EncryptMultiply")

In [53]:
hpc3_em_df = get_worker_duration("charlie", worker_dfs, "EncryptMultiply")

In [54]:
hpc1_em_df['Time'][1:].mean()

72.92684103250504

In [55]:
hpc2_em_df['Time'][1:].mean()

85.04863657951356

In [56]:
hpc3_em_df['Time'][1:].mean()

82.33616433143615

In [57]:
hpc1_em_df

,WorkerID,EventType,EventState,Time
0,alice,EncryptMultiply,duration,211.737648
1,alice,EncryptMultiply,duration,210.702761
2,alice,EncryptMultiply,duration,221.539311
3,alice,EncryptMultiply,duration,211.035815
4,alice,EncryptMultiply,duration,210.830552
5,alice,EncryptMultiply,duration,210.901063
6,alice,EncryptMultiply,duration,210.987177
7,alice,EncryptMultiply,duration,211.577678
8,alice,EncryptMultiply,duration,210.910632
9,alice,EncryptMultiply,duration,212.308907


In [58]:
hpc2_em_df

,WorkerID,EventType,EventState,Time
0,bob,EncryptMultiply,duration,233.724081
1,bob,EncryptMultiply,duration,233.994982
2,bob,EncryptMultiply,duration,216.604090
3,bob,EncryptMultiply,duration,233.318352
4,bob,EncryptMultiply,duration,232.208992
5,bob,EncryptMultiply,duration,233.171974
6,bob,EncryptMultiply,duration,233.528935
7,bob,EncryptMultiply,duration,233.360581
8,bob,EncryptMultiply,duration,232.415484
9,bob,EncryptMultiply,duration,234.987400


## EncryptParameter Duration

In [59]:
def get_encrypt_parameter_duration(worker_id, worker_dfs):
    worker_df = worker_dfs[worker_id]
    worker_ep_df = worker_df[worker_df["EventType"].str.contains('EncryptParameter', regex=False)]
    worker_ep_df = worker_ep_df.reset_index()
    
    del worker_ep_df["index"]
    del worker_ep_df["ID"]
    
    return worker_ep_df
    

In [60]:
# hpc1_ep_df = get_encrypt_parameter_duration("alice", worker_dfs)
hpc1_ep_df = get_worker_duration("alice", worker_dfs, "EncryptParameter")
hpc2_ep_df = get_worker_duration("bob", worker_dfs, "EncryptParameter")
hpc3_ep_df = get_worker_duration("charlie", worker_dfs, "EncryptParameter")

In [61]:
hpc1_ep_df[0:8]

,WorkerID,EventType,EventState,Time
0,alice,EncryptParameter0,duration,0.009029
1,alice,EncryptParameter1,duration,0.005699
2,alice,EncryptParameter2,duration,0.476333
3,alice,EncryptParameter3,duration,0.004884
4,alice,EncryptParameter4,duration,130.180951
5,alice,EncryptParameter5,duration,0.169991
6,alice,EncryptParameter6,duration,80.360515
7,alice,EncryptParameter7,duration,0.019217


In [81]:
mean_values = []
for i in range(8):
    mean_value = hpc1_ep_df[hpc1_ep_df["EventType"] == "EncryptParameter"+str(i)]["Time"].mean()
    mean_values.append(mean_value)
    print(str(i)+":", mean_value)

0: 0.00933792477562314
1: 0.005080416089012509
2: 0.4695088409242176
3: 0.047009797323317754
4: 130.38696922574724
5: 0.17132852191016787
6: 79.92306349391029
7: 0.021012158620925175


In [82]:
sum(mean_values)

211.0333103793008

In [85]:
for i in range(8):
    mean_value = hpc3_ep_df[hpc1_ep_df["EventType"] == "EncryptParameter"+str(i)]["Time"].mean()
    print(str(i)+":", mean_value)

0: 7.9473661581675215
1: 0.0047116393134707495
2: 0.46585534867786227
3: 0.005139362244378953
4: 120.36462834903172
5: 0.1791796116601853
6: 90.33547802198501
7: 0.015599489212036133


In [84]:
for i in range(8):
    mean_value = hpc3_ep_df[hpc1_ep_df["EventType"] == "EncryptParameter"+str(i)]["Time"].mean()
    print(str(i)+":", mean_value)

0: 7.9473661581675215
1: 0.0047116393134707495
2: 0.46585534867786227
3: 0.005139362244378953
4: 120.36462834903172
5: 0.1791796116601853
6: 90.33547802198501
7: 0.015599489212036133


In [66]:
hpc1_ep_df["Time"][0:8]

0      0.009029
1      0.005699
2      0.476333
3      0.004884
4    130.180951
5      0.169991
6     80.360515
7      0.019217
Name: Time, dtype: float64

In [70]:
hpc2_ep_df["Time"][0:8]

0      8.251601
1      0.004555
2      0.472421
3      0.005668
4    142.546268
5      0.168032
6     81.985369
7      0.007378
Name: Time, dtype: float64

In [67]:
hpc1_ep_df["Time"].mean()

21.115078750110808

In [68]:
hpc2_ep_df["Time"].mean()

23.066314332825797

In [69]:
hpc3_ep_df["Time"].mean()

21.94343036129361

## MultiplyParameter Duration

In [70]:
def get_multiply_parameter_duration(worker_id, worker_dfs):
    worker_df = worker_dfs[worker_id]
    worker_ep_df = worker_df[worker_df["EventType"].str.contains('MultiplyParameter', regex=False)]
    worker_ep_df = worker_ep_df.reset_index()
    
    del worker_ep_df["index"]
    del worker_ep_df["ID"]
    
    return worker_ep_df

In [71]:
# hpc1_mp_df = get_multiply_parameter_duration("alice", worker_dfs)
hpc1_mp_df = get_worker_duration("alice", worker_dfs, "MultiplyParameter")
hpc2_mp_df = get_worker_duration("bob", worker_dfs, "MultiplyParameter")
hpc3_mp_df = get_worker_duration("charlie", worker_dfs, "MultiplyParameter")

In [72]:
hpc1_mp_df[:8]["Time"].sum()

3.198965549468994

In [73]:
hpc1_mp_df["Time"].mean()

0.4327079852422078

In [74]:
hpc2_mp_df["Time"].mean()

0.4213984591620309

In [75]:
hpc3_mp_df["Time"].mean()

0.9123388494764055

## RetrieveTime duration

In [76]:
def get_coord_retrieve_duration(coord_df, worker_id):
    coord_event_df = coord_df[coord_df["EventType"].str.contains("RetrieveTime", regex=False)]
    coord_event_df = coord_event_df[coord_event_df["ID"] == worker_id]
    coord_event_df = coord_event_df.reset_index()
    
    del coord_event_df['index']
    
    return coord_event_df

In [77]:
hpc1_ret_df = get_coord_retrieve_duration(coord_df, "alice")

In [78]:
hpc2_ret_df = get_coord_retrieve_duration(coord_df, "bob")

In [79]:
hpc3_ret_df = get_coord_retrieve_duration(coord_df, "charlie")

In [80]:
hpc1_ret_df["Time"][1:].mean()

0.019368553161621095

In [81]:
hpc2_ret_df["Time"][1:].mean()

0.018344151973724365

In [82]:
hpc3_ret_df["Time"][1:].mean()

0.018260812759399413

## ShareSend Duration

In [83]:
def calculate_share_send_duration(worker_id, worker_dfs, nr_parameters):
    worker_df = worker_dfs[worker_id]
    worker_send_share_df = worker_df[worker_df["EventType"].str.contains('SendShare', regex=False)]
    
    worker_send_share_start_df = worker_send_share_df[worker_send_share_df["EventState"] == 'start']
    worker_send_share_start_df = worker_send_share_start_df.reset_index()
    worker_send_share_start_df = worker_send_share_start_df[["WorkerID", "EventType", "ID", "Time"]]
    worker_send_share_start_df.columns = ["WorkerID", "EventType", "ID", "StartTime"]
    
    worker_send_share_end_df = worker_send_share_df[worker_send_share_df["EventState"] == 'end']
    worker_send_share_end_df = worker_send_share_end_df.reset_index()
    
    send_share_df_cat = pd.concat([worker_send_share_start_df, worker_send_share_end_df["Time"]], axis=1)
    send_share_df_cat.columns = ["WorkerID", "EventType", "ID", "StartTime", "EndTime"]
    send_share_df_cat["Duration"] = send_share_df_cat.apply(lambda x: x["EndTime"] - x["StartTime"], axis=1)
    
    parameter_index = []
    nr_worker = len(worker_dfs)
    p_index = -1

    for i in range(len(send_share_df_cat)):
        tmp_index = (i%nr_worker)
        if tmp_index == 0:
            p_index += 1
            p_index = p_index % nr_parameters
        parameter_index.append(p_index)
        
    send_share_df_cat.insert(2, "p_idx", parameter_index)
    obj_id_list = send_share_df_cat.apply(lambda x: x["EventType"].split("_")[1], axis=1)
    send_share_df_cat.insert(2, "obj_id", obj_id_list)
    
    del send_share_df_cat["EventType"]
    send_share_df_cat.insert(1, "EventType", "SendShare")
    
    return send_share_df_cat


In [84]:
alice_send_share_df_cat = calculate_share_send_duration("alice", worker_dfs, 8)
bob_send_share_df_cat = calculate_share_send_duration("bob", worker_dfs, 8)
charlie_send_share_df_cat = calculate_share_send_duration("charlie", worker_dfs, 8)

In [85]:
alice_send_share_df_cat

,WorkerID,EventType,obj_id,p_idx,ID,StartTime,EndTime,Duration
0,alice,SendShare,7611270575,0,alice,1.603614e+09,1.603614e+09,0.000829
1,alice,SendShare,61904364723,0,bob,1.603614e+09,1.603614e+09,0.003386
2,alice,SendShare,86059624810,0,charlie,1.603614e+09,1.603614e+09,0.003282
3,alice,SendShare,17410712336,1,alice,1.603614e+09,1.603614e+09,0.000545
4,alice,SendShare,33161346880,1,bob,1.603614e+09,1.603614e+09,0.001775
...,...,...,...,...,...,...,...,...
625,alice,SendShare,30995103264,0,bob,1.603616e+09,1.603616e+09,0.012489
626,alice,SendShare,48833848601,0,charlie,1.603616e+09,1.603616e+09,0.022400
627,alice,SendShare,88464557323,1,alice,1.603616e+09,1.603616e+09,0.000906
628,alice,SendShare,44445403221,1,bob,1.603616e+09,1.603616e+09,0.001732


In [86]:
bob_send_share_df_cat

,WorkerID,EventType,obj_id,p_idx,ID,StartTime,EndTime,Duration
0,bob,SendShare,36203251608,0,alice,1.603614e+09,1.603614e+09,8.448553
1,bob,SendShare,55651927346,0,bob,1.603614e+09,1.603614e+09,0.001036
2,bob,SendShare,63878039959,0,charlie,1.603614e+09,1.603614e+09,0.004267
3,bob,SendShare,74822626216,1,alice,1.603614e+09,1.603614e+09,0.001704
4,bob,SendShare,42676332037,1,bob,1.603614e+09,1.603614e+09,0.000607
...,...,...,...,...,...,...,...,...
625,bob,SendShare,11686709192,0,bob,1.603616e+09,1.603616e+09,0.001081
626,bob,SendShare,66087521754,0,charlie,1.603616e+09,1.603616e+09,0.014263
627,bob,SendShare,82060761398,1,alice,1.603616e+09,1.603616e+09,0.002541
628,bob,SendShare,6133340990,1,bob,1.603616e+09,1.603616e+09,0.000944


In [87]:
charlie_send_share_df_cat

,WorkerID,EventType,obj_id,p_idx,ID,StartTime,EndTime,Duration
0,charlie,SendShare,84232349682,0,alice,1.603614e+09,1.603614e+09,7.354068
1,charlie,SendShare,8936010956,0,bob,1.603614e+09,1.603614e+09,0.006361
2,charlie,SendShare,46379652235,0,charlie,1.603614e+09,1.603614e+09,0.000968
3,charlie,SendShare,28758671673,1,alice,1.603614e+09,1.603614e+09,0.002265
4,charlie,SendShare,58929676202,1,bob,1.603614e+09,1.603614e+09,0.001569
...,...,...,...,...,...,...,...,...
625,charlie,SendShare,12398153,0,bob,1.603616e+09,1.603616e+09,0.015157
626,charlie,SendShare,51605127601,0,charlie,1.603616e+09,1.603616e+09,0.000778
627,charlie,SendShare,22153551532,1,alice,1.603616e+09,1.603616e+09,0.001763
628,charlie,SendShare,88084886423,1,bob,1.603616e+09,1.603616e+09,0.001354


## ShareSend Send Time

In [88]:
def calculate_share_send_time(sender_id, receiver_id, worker_dfs, nr_parameter):
    sender_df = worker_dfs[sender_id]
    receiver_df = worker_dfs[receiver_id]
    
    sender_send_share_df = sender_df[sender_df["EventType"].str.contains('SendShare', regex=False)]
    receiver_get_share_df = receiver_df[receiver_df["EventType"].str.contains('GetShare', regex=False)]
    
    send_share_start_df = sender_send_share_df[(sender_send_share_df["ID"] == receiver_id) & (sender_send_share_df["EventState"] == 'start')]
    get_share_df = receiver_get_share_df[receiver_get_share_df["ID"] == sender_id]
    send_share_start_df = send_share_start_df.reset_index()
    get_share_df = get_share_df.reset_index()
    
    del send_share_start_df['index']
    del send_share_start_df['EventState']
    
    send_time_df = pd.concat([send_share_start_df, get_share_df["Time"]], axis=1)
    send_time_df.columns = ["WorkerID", "EventType", "ID", "StartTime", "RecvTime"]
    send_time_df["SendTime"] = send_time_df.apply(lambda x: x["RecvTime"] - x["StartTime"], axis=1)
    
    nr_parameter = 8
    parameter_index = []

    for i in range(len(send_time_df)):
        tmp_index = (i%nr_parameter)
        parameter_index.append(tmp_index)
        
    send_time_df.insert(2, "p_idx", parameter_index)
    
    obj_id_list = send_time_df.apply(lambda x: x["EventType"].split("_")[1], axis=1)
    send_time_df.insert(2, "obj_id", obj_id_list)
    
    del send_time_df["EventType"]
    send_time_df.insert(1, "EventType", "SendShare")
    
    return send_time_df

In [89]:
send_time_df = calculate_share_send_time("alice", "bob", worker_dfs, 8)

In [90]:
send_time_df

,WorkerID,EventType,obj_id,p_idx,ID,StartTime,RecvTime,SendTime
0,alice,SendShare,61904364723,0,bob,1.603614e+09,1.603614e+09,-0.016973
1,alice,SendShare,33161346880,1,bob,1.603614e+09,1.603614e+09,-0.018621
2,alice,SendShare,7896894442,2,bob,1.603614e+09,1.603614e+09,0.026093
3,alice,SendShare,82748437727,3,bob,1.603614e+09,1.603614e+09,0.023020
4,alice,SendShare,6354039845,4,bob,1.603614e+09,1.603614e+09,19.603234
...,...,...,...,...,...,...,...,...
205,alice,SendShare,95774474162,5,bob,1.603616e+09,1.603616e+09,0.936910
206,alice,SendShare,97485782738,6,bob,1.603616e+09,1.603616e+09,18.748248
207,alice,SendShare,89049343302,7,bob,1.603616e+09,1.603616e+09,-0.017776
208,alice,SendShare,30995103264,0,bob,1.603616e+09,1.603616e+09,-0.008539


## AllWorkersTrainingTime Duration

In [91]:
coord_awtt_df = get_coord_duration(coord_df, "AllWorkersTrainingTime")

In [92]:
coord_awtt_df['Time'][1:].mean()

100.00995072126389

In [93]:
coord_awtt_df

,EventType,EventState,Time
0,AllWorkersTrainingTime,duration,102.239992
1,AllWorkersTrainingTime,duration,107.253419
2,AllWorkersTrainingTime,duration,96.193710
3,AllWorkersTrainingTime,duration,106.637033
4,AllWorkersTrainingTime,duration,97.999094
5,AllWorkersTrainingTime,duration,96.269811
6,AllWorkersTrainingTime,duration,107.040473
7,AllWorkersTrainingTime,duration,99.219020
8,AllWorkersTrainingTime,duration,97.069740
9,AllWorkersTrainingTime,duration,99.601255


## AggregationTime

In [94]:
coord_aggt_df = get_coord_duration(coord_df, "AggregationTime")

In [95]:
coord_aggt_df["Time"][1:].mean()

0.22990245819091798

In [96]:
coord_aggt_df

,EventType,EventState,Time
0,AggregationTime,duration,0.245026
1,AggregationTime,duration,0.228312
2,AggregationTime,duration,0.216894
3,AggregationTime,duration,0.232062
4,AggregationTime,duration,0.215400
5,AggregationTime,duration,0.232104
6,AggregationTime,duration,0.231468
7,AggregationTime,duration,0.244203
8,AggregationTime,duration,0.219201
9,AggregationTime,duration,0.229891


## DecryptionTime

In [97]:
coord_decrypt_df = get_coord_duration(coord_df, "DecryptionTime")

In [98]:
coord_decrypt_df['Time'][1:].mean()

7.425192427635193

In [99]:
coord_decrypt_df

,EventType,EventState,Time
0,DecryptionTime,duration,7.546542
1,DecryptionTime,duration,7.498196
2,DecryptionTime,duration,7.235712
3,DecryptionTime,duration,7.486921
4,DecryptionTime,duration,7.378316
5,DecryptionTime,duration,7.479644
6,DecryptionTime,duration,7.512646
7,DecryptionTime,duration,7.537995
8,DecryptionTime,duration,7.534792
9,DecryptionTime,duration,7.465533


## RoundTime

In [100]:
coord_round_time_df = get_coord_duration(coord_df, "RoundTime")

In [101]:
coord_round_time_df["Time"][1:].mean()

107.74317790269852

In [100]:
coord_round_time_df

,EventType,EventState,Time
0,RoundTime,duration,19.145947
1,RoundTime,duration,18.873726
2,RoundTime,duration,19.465924
3,RoundTime,duration,19.579201
4,RoundTime,duration,19.174881
5,RoundTime,duration,18.636325
6,RoundTime,duration,19.162090
7,RoundTime,duration,18.452327
8,RoundTime,duration,19.007770
9,RoundTime,duration,19.201967
